<a href="https://colab.research.google.com/github/gabrielkerr/deep_learning_fall_2018/blob/master/Keras_Content_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
# Installing packages and cloning git repos
!pip install music21
!pip install h5py
!git clone https://github.com/Skuldur/Classical-Piano-Composer.git

    100% |████████████████████████████████| 18.5MB 1.7MB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/7b/21/95/d396f231b8095f30aba2a1fbffbc2411fb22eb4e611ddbed57
Successfully built music21
Cloning into 'Classical-Piano-Composer'...
remote: Enumerating objects: 150, done.
remote: Total 150 (delta 0), reused 0 (delta 0), pack-reused 150
Receiving objects: 100% (150/150), 117.92 MiB | 30.40 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [0]:
# switch directories to the cloned repo with defined helper functions
import os
os.chdir('Classical-Piano-Composer')

In [4]:
# imports
import keras
from keras import backend as K
import tensorflow as tf
import music21
import h5py
import os
from lstm import get_notes, prepare_sequences
import numpy as np

Using TensorFlow backend.


# Get Features

In [5]:
# Get notes from midi files
notes = get_notes()

Parsing midi_songs/Kingdom_Hearts_Traverse_Town.mid
Parsing midi_songs/ff4_piano_collections-main_theme.mid
Parsing midi_songs/sera_.mid
Parsing midi_songs/Eternal_Harvest.mid
Parsing midi_songs/ff4-airship.mid
Parsing midi_songs/great_war.mid
Parsing midi_songs/relmstheme-piano.mid
Parsing midi_songs/ff4-fight1.mid
Parsing midi_songs/ff1battp.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/roseofmay-piano.mid
Parsing midi_songs/ultimafro.mid
Parsing midi_songs/FF4.mid
Parsing midi_songs/ViviinAlexandria.mid
Parsing midi_songs/pkelite4.mid
Parsing midi_songs/In_Zanarkand.mid
Parsing midi_songs/Ff7-One_Winged.mid
Parsing midi_songs/FF6epitaph_piano.mid
Parsing midi_songs/Life_Stream.mid
Parsing midi_songs/mining.mid
Parsing midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
Parsing midi_songs/z_aeristhemepiano.mid
Parsing midi_songs/bcm.mid
Parsing midi_songs/VincentPiano.mid
Parsing midi_songs/Still_Alive-1.mid
Parsing midi_songs/ff7-mainmidi.mid
Parsing midi_songs/

In [0]:
# Get count of unique "vocabulary" (notes)
n_vocab = len(set(notes))

In [0]:
# Extract sequences of 100 notes from midis
network_input, network_output = prepare_sequences(notes, n_vocab)

# Naive Loss Function

Content loss is defined as the norm between `input_sequence` - `output_sequence`.

Here we perform content loss between the 1st and 2nd input sequences to show functionality of the loss function, but ideally a sequence of 100 notes would be generated from the trained network and then compared with the input sequence.

In [0]:
def content_loss(input_sequence, output_sequence):
  return np.linalg.norm(input_sequence - output_sequence)

In [29]:
content_loss(network_input[0], network_input[1])

4.154247546068616

In [0]:
""" This module generates notes for a midi file using the
  trained neural network """
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

def generate(idx):
  """ Generate a piano midi file """
  #load the notes used to train the model
  notes = get_notes()

  # Get all pitch names
  pitchnames = sorted(set(item for item in notes))
  # Get all pitch names
  n_vocab = len(set(notes))

  network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
  model = create_network(normalized_input, n_vocab)
  prediction_output, pattern = generate_notes(model, network_input, idx, pitchnames, n_vocab)
  create_midi(prediction_output)
  
  return pattern


def prepare_sequences(notes, pitchnames, n_vocab):
  """ Prepare the sequences used by the Neural Network """
  # map between notes and integers and back
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

  sequence_length = 100
  network_input = []
  output = []
  for i in range(0, len(notes) - sequence_length, 1):
      sequence_in = notes[i:i + sequence_length]
      sequence_out = notes[i + sequence_length]
      network_input.append([note_to_int[char] for char in sequence_in])
      output.append(note_to_int[sequence_out])

  n_patterns = len(network_input)

  # reshape the input into a format compatible with LSTM layers
  normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
  # normalize input
  normalized_input = normalized_input / float(n_vocab)

  return (network_input, normalized_input)

def create_network(network_input, n_vocab):
  """ create the structure of the neural network """
  model = Sequential()
  model.add(LSTM(
      512,
      input_shape=(network_input.shape[1], network_input.shape[2]),
      return_sequences=True
  ))
  model.add(Dropout(0.3))
  model.add(LSTM(512, return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(512))
  model.add(Dense(256))
  model.add(Dropout(0.3))
  model.add(Dense(n_vocab+1))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

  # Load the weights to each node
  model.load_weights('new_weights.hdf5')

  return model

def generate_notes(model, network_input, idx, pitchnames, n_vocab):
  """ Generate notes from the neural network based on a sequence of notes """
  # pick a random sequence from the input as a starting point for the prediction
  #start = numpy.random.randint(0, len(network_input)-1)
  start = idx

  int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

  pattern = network_input[start]
  prediction_output = []

  # generate 500 notes
  #for note_index in range(500):
  for note_index in range(100):

      prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
      prediction_input = prediction_input / float(n_vocab)

      prediction = model.predict(prediction_input, verbose=0)

      index = numpy.argmax(prediction)
      result = int_to_note[index]
      prediction_output.append(result)

      pattern.append(index)
      pattern = pattern[1:len(pattern)]

  return prediction_output, pattern

def create_midi(prediction_output):
  """ convert the output from the prediction to notes and create a midi file
      from the notes """
  offset = 0
  output_notes = []

  # create note and chord objects based on the values generated by the model
  for pattern in prediction_output:
      # pattern is a chord
      if ('.' in pattern) or pattern.isdigit():
          notes_in_chord = pattern.split('.')
          notes = []
          for current_note in notes_in_chord:
              new_note = note.Note(int(current_note))
              new_note.storedInstrument = instrument.Piano()
              notes.append(new_note)
          new_chord = chord.Chord(notes)
          new_chord.offset = offset
          output_notes.append(new_chord)
      # pattern is a note
      else:
          new_note = note.Note(pattern)
          new_note.offset = offset
          new_note.storedInstrument = instrument.Piano()
          output_notes.append(new_note)

      # increase offset each iteration so that notes do not stack
      offset += 0.5

  midi_stream = stream.Stream(output_notes)

  midi_stream.write('midi', fp='test_output.mid')

In [65]:
preds = generate(0)

Parsing midi_songs/Kingdom_Hearts_Traverse_Town.mid
Parsing midi_songs/ff4_piano_collections-main_theme.mid
Parsing midi_songs/sera_.mid
Parsing midi_songs/Eternal_Harvest.mid
Parsing midi_songs/ff4-airship.mid
Parsing midi_songs/great_war.mid
Parsing midi_songs/relmstheme-piano.mid
Parsing midi_songs/ff4-fight1.mid
Parsing midi_songs/ff1battp.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/roseofmay-piano.mid
Parsing midi_songs/ultimafro.mid
Parsing midi_songs/FF4.mid
Parsing midi_songs/ViviinAlexandria.mid
Parsing midi_songs/pkelite4.mid
Parsing midi_songs/In_Zanarkand.mid
Parsing midi_songs/Ff7-One_Winged.mid
Parsing midi_songs/FF6epitaph_piano.mid
Parsing midi_songs/Life_Stream.mid
Parsing midi_songs/mining.mid
Parsing midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
Parsing midi_songs/z_aeristhemepiano.mid
Parsing midi_songs/bcm.mid
Parsing midi_songs/VincentPiano.mid
Parsing midi_songs/Still_Alive-1.mid
Parsing midi_songs/ff7-mainmidi.mid
Parsing midi_songs/

In [68]:
# Content loss between normalized generated sequence and normalized input sequence
content_loss(network_input[0], np.array(preds)[:100]/n_vocab)


37.62260100803737

In [0]:
# Load trained model
from keras.layers import LSTM, Dropout, Dense, Activation
from keras import Sequential

model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('new_weights.hdf5')

In [11]:
len(model.get_weights()[0])

359

In [18]:
import tensorflow as tf
with tf.Session() as sess:
  init = tf.global_variables_initializer()
  sess.run(init)
  v = sess.run(model.weights[0])
  print(v) 
  assign_op = model.weights[0].assign(np.zeros(v.shape))
  sess.run(assign_op)
  print(model.weights[0].eval())

[[ 0.01884716 -0.035212   -0.04476875 ... -0.03120076 -0.00696015
  -0.01515903]]
[[0. 0. 0. ... 0. 0. 0.]]
